## Importamos librerias

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re

## Importamos la URL y la convertimos a texto

In [2]:
url = "https://sports.williamhill.es/betting/es-es/football/competitions/OB_TY338/Espana-LaLiga-Santander/outrights/OB_MG101671/Descendera"
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")

## Obtenemos las cuotas limpias, y las metemos en un DataFrame 

In [3]:
def cuotas1(i):
    table = soup.find_all("td")[i]
    table2 = str(table.find_all("button")[0])
    splitted = table2.split("span>")[1]
    splitted2 = splitted.split("</")
    return splitted2[0]

In [4]:
cuotas = []
for i in range(20):
    cuotas.append(cuotas1(i))

In [5]:
cuotas2 = []
for i in cuotas:
    cuotas2.append(i.split("/"))

In [6]:
x = pd.DataFrame(cuotas2)
x[0] = [int(r) for r in x[0]]
x[1] = [int(r) for r in x[1]]
x[3] = round((x[0]/x[1])+1,2)
x = x.drop(columns=[0,1])

## Obtenemos los equipos limpios y los metemos en un DataFrame

In [7]:
def equipos1(i):
    table = soup.find_all("td")[i]
    table2 = table.find_all("span")[0]
    table3 = table2.find_all("span")[0]
    table4 = str(table3.find_all("span")[1])
    table5 = table4.split(">")[1]
    table6 = table5.split("<")[0]
    return table6

In [8]:
equipos = []
for i in range(20):
    equipos.append(equipos1(i))

In [9]:
y = pd.DataFrame(equipos)

## Concatenamos ambas tablas, y corregimos los errores de texto

In [10]:
x["equipos"]= y[0]

In [11]:
x = x.rename(columns={3:"Cuotas"})

In [12]:
x["Equipos"] = [re.sub("Ã©","é",i) for i in x["equipos"]]

In [13]:
x["Equipos"] = [re.sub("Ã¡","á",i) for i in x["Equipos"]]

In [14]:
x = x.drop(columns=["equipos"])

In [15]:
x.to_csv("Descenderán.csv")